## Data cleaning and feature engineering

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/voter-survey-december16.csv')

In [12]:
pattern = '(imiss_).{1}(_2016)'
def return_imiss_cols(df):
    imiss_cols = []
    for col in df.columns:
        if re.search(pattern, col):
            imiss_cols.append(col)
    return imiss_cols

imiss_col_list = return_imiss_cols(df)

target_col_list = 'presvote16post_2016', ''
        

['imiss_a_2016',
 'imiss_b_2016',
 'imiss_c_2016',
 'imiss_d_2016',
 'imiss_e_2016',
 'imiss_f_2016',
 'imiss_g_2016',
 'imiss_h_2016',
 'imiss_i_2016',
 'imiss_j_2016',
 'imiss_k_2016',
 'imiss_l_2016',
 'imiss_m_2016',
 'imiss_n_2016',
 'imiss_o_2016',
 'imiss_p_2016',
 'imiss_q_2016',
 'imiss_r_2016',
 'imiss_s_2016',
 'imiss_t_2016',
 'imiss_u_2016',
 'imiss_x_2016',
 'imiss_y_2016']

In [ ]:
cols = ['presvote16post_2016', 'imiss_u_2016', 'weight']
df = df[cols]
df = df.fillna(0)
# convert all categorical cols to strings for dummy encoding
df.imiss_u_2016 = df.imiss_u_2016.astype(str)
df = pd.get_dummies(df)

In [ ]:
df.corr()['presvote16post_2016']